In [1]:
# Import required libraries
import rasterio
from rasterio import features
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from IPython.display import display
import folium

In [29]:
# Set the western boundary of the study area
minx = 252400 
maxy = 286000

# Define paths to LiDAR cover raster files
cover_2019_path = 'data/raster/lidar_products/2019_lidar_cover.tif'
cover_2020_path = 'data/raster/lidar_products/2020_lidar_cover.tif'

def raster_to_polygon_with_minx(raster_path, minx, maxy):
    """
    Convert a raster to polygons, cropping at specified western boundary (minx) and northern boundary (maxy)
    
    Args:
        raster_path (str): Path to the input raster file
        minx (float): Minimum x coordinate to crop the raster
        maxy (float): Maximum y coordinate to crop the raster
        
    Returns:
        GeoDataFrame: Polygons derived from the raster
    """
    with rasterio.open(raster_path) as src:
        raster_array = src.read(1)
        
        # Adjust extents based on minx and maxy
        xmin, ymin, xmax, ymax = src.bounds
        
        x_offset = 0
        y_offset = 0
        
        if xmin < minx:
            # Calculate offset and crop raster
            x_offset = int((minx - xmin) / src.res[0])
            
        if ymax > maxy:
            y_offset = int((ymax - maxy) / abs(src.res[1]))
            
        # Move raster cropping to after new transform calculation
        
        # Calculate new transform
        new_xmin = xmin + x_offset * src.res[0] if xmin < minx else xmin
        new_ymax = ymax - y_offset * abs(src.res[1]) if ymax > maxy else ymax
        
        new_transform = rasterio.transform.from_origin(new_xmin, new_ymax, src.res[0], src.res[1])
        
        # Crop raster here
        raster_array = raster_array[y_offset:, x_offset:]
        
        # Create mask for valid data (values <= 100)
        mask = raster_array <= 100
        
        # Generate shapes from the mask
        shapes = features.shapes(mask.astype('int16'), mask=mask, transform=new_transform)
        
        # Convert shapes to a GeoDataFrame
        polygons = []
        for geom, value in shapes:
            if value == 1:
                polygons.append(Polygon(geom['coordinates'][0]))
        
        gdf = gpd.GeoDataFrame({'geometry': polygons}, crs=src.crs)
        return gdf

# Convert both years' rasters to polygons
gdf_2019 = raster_to_polygon_with_minx(cover_2019_path, minx, maxy)
gdf_2020 = raster_to_polygon_with_minx(cover_2020_path, minx, maxy)

# Convert to WGS84 for web mapping
gdf_2019_4326 = gdf_2019.to_crs(epsg=4326)
gdf_2020_4326 = gdf_2020.to_crs(epsg=4326)

# Create a folium map centered on the data
combined_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(list(gdf_2019_4326.geometry) + list(gdf_2020_4326.geometry)))
centroid = combined_gdf.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12, tiles="OpenStreetMap")

# Add satellite imagery base layer
tiles = folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite')
tiles.add_to(m)

# Add 2019 coverage (green)
folium.GeoJson(gdf_2019_4326, style_function=lambda feature: {
    'fillColor': 'green',
    'color': 'green',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

# Add 2020 coverage (blue)
folium.GeoJson(gdf_2020_4326, style_function=lambda feature: {
    'fillColor': 'blue',
    'color': 'blue',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

display(m)

/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_73683/1830299259.py:73: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = combined_gdf.geometry.unary_union.centroid


In [30]:
intersection = gpd.overlay(gdf_2019, gdf_2020, how='intersection')
gdf_2019_only = gpd.overlay(gdf_2019, gdf_2020, how='difference')

intersection_4326 = intersection.to_crs(epsg=4326)
gdf_2019_only_4326 = gdf_2019_only.to_crs(epsg=4326)

combined_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(list(intersection_4326.geometry) + list(gdf_2019_only_4326.geometry)))
centroid = combined_gdf.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12, tiles="OpenStreetMap")

tiles = folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite')
tiles.add_to(m)


folium.GeoJson(intersection_4326, style_function=lambda feature: {
    'fillColor': 'purple',
    'color': 'purple',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

folium.GeoJson(gdf_2019_only_4326, style_function=lambda feature: {
    'fillColor': 'green',
    'color': 'green',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

display(m)


/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_73683/110592257.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = combined_gdf.geometry.unary_union.centroid


In [31]:
all_path = 'data/vector/all_areas.geojson'
test_path = 'data/vector/test_areas.geojson'
train_path = 'data/vector/train_areas.geojson'

# Create union of test and train areas
all_areas = gpd.GeoDataFrame(pd.concat([intersection, gdf_2019_only], ignore_index=True))

all_areas.to_file(all_path, driver='GeoJSON')
intersection.to_file(test_path, driver='GeoJSON')
gdf_2019_only.to_file(train_path, driver='GeoJSON')

In [32]:
path_to_raster_template = 'data/raster/lidar_products/2019_lidar_cover.tif'

with rasterio.open(path_to_raster_template) as src:
    
    # Get the raster's bounding box in its native CRS
    left, bottom, right, top = src.bounds
    
    # Get the raster's transform
    transform = src.transform
    
    # Get the raster's width and height
    width = src.width
    height = src.height
    
    # Get the spatial resolution
    resolution_x, resolution_y = src.res
    print(f"Resolution: {resolution_x}, {resolution_y}")
    # Create a list to hold the cell centroids
    cell_centroids = []
    
    # Loop through each cell in the raster
    for row in range(height):
        for col in range(width):
            # Calculate the coordinates of the cell's corners
            x1, y1 = transform * (col, row)
            x2, y2 = transform * (col + 1, row)
            x3, y3 = transform * (col + 1, row + 1)
            x4, y4 = transform * (col, row + 1)
            
            # Create a polygon from the cell's corners
            polygon = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)])
            
            # Get the centroid of the polygon
            centroid = polygon.centroid
            
            # Add the centroid to the list
            cell_centroids.append(centroid)
            
    # Create a GeoDataFrame from the cell centroids
    raster_cell_centroids = gpd.GeoDataFrame(geometry=cell_centroids, crs=src.crs)


Resolution: 29.26080000028014, 29.26080000028014


In [33]:
def subset_points(points_gdf, area_gdf, area_name):
    """Subsets points within a given area and saves to a geojson.

    Args:
        points_gdf (geopandas.GeoDataFrame): GeoDataFrame of points.
        area_gdf (geopandas.GeoDataFrame): GeoDataFrame of area polygon.
        area_name (str): Name of the area (for saving).
    """
    points_in_area = gpd.sjoin(points_gdf, area_gdf, predicate='within')
    print(f'{area_name} points: {len(points_in_area)}')
    points_in_area = points_in_area[['geometry']]
    points_in_area.to_file(f'data/vector/{area_name}_points.geojson', driver='GeoJSON')
    return points_in_area

train_area = gpd.read_file('data/vector/train_areas.geojson')
test_area = gpd.read_file('data/vector/test_areas.geojson')

train_points = subset_points(raster_cell_centroids, train_area, 'train')
test_points = subset_points(raster_cell_centroids, test_area, 'test')


train points: 36233
test points: 7952
